In [ ]:
let input = 
  File.ReadAllLines "input.txt" 
  |> Seq.map (fun s -> s.ToCharArray() |> Array.map (fun c -> int c - int '0'))

let collapseList l =
  l |> Array.map (fun (x,y) -> x + y)

let mostCommonBits = 
  input
  |> Seq.map (Array.map (fun x -> x * 2 - 1))
  |> Seq.reduce (fun a b -> collapseList <| (Array.zip a b))
  |> Array.map (function x when x > 0 -> 1 | _ -> 0)

let flip x = (x + 1) % 2

let binaryToInt: (int seq -> int) = Seq.fold (fun a x -> 2 * a + x) 0

let gamma = mostCommonBits |> binaryToInt
let epsilon = mostCommonBits |> Array.map flip |> binaryToInt

gamma * epsilon

1082324

In [ ]:
type Trie =
  | Node of zero: Trie * one: Trie
  | ZeroOnly of Trie
  | OneOnly of Trie
  | Leaf

type Bit = 
  | One
  | Zero

let toBit = function 1 -> One | 0 -> Zero | _ -> failwith "err"

let rec add t = function
  | [] -> t
  | Zero :: rest -> 
    match t with 
    | (ZeroOnly z | (Leaf as z)) -> ZeroOnly (add z rest)
    | OneOnly o -> Node(add Leaf rest, o)
    | Node(z,o) -> Node(add z rest, o)   
  | One :: rest -> 
    match t with 
    | (OneOnly o | (Leaf as o)) -> OneOnly(add o rest)
    | ZeroOnly z -> Node(z, add Leaf rest)
    | Node(z,o) -> Node(z, add o rest)

let inputTrie = 
  input 
  |> Seq.map (Seq.map toBit >> Seq.toList)
  |> Seq.fold add Leaf

#load "../Utils.fsx"
open Utils

let rec count = memoize (function
  | Leaf -> 1
  | OneOnly t | ZeroOnly t -> count t
  | Node(z,o) -> count z + count o
)

let rec findRating cond = 
  function
  | Node(z, o) -> if cond (count z) (count o) then (0 :: findRating cond z) else (1 :: findRating cond o)
  | OneOnly t -> 1 :: findRating cond t
  | ZeroOnly t -> 0 :: findRating cond t
  | Leaf -> []

let findO2 = findRating (>) >> binaryToInt
let findCO2 = findRating (<=) >> binaryToInt

(findO2 inputTrie, findCO2 inputTrie) ||> (*)


1353024